## Visualisation des résultats

In [3]:
# importation of modules
import cpnet
import numpy as np
import math
import pandas as pd
import networkx as nx
from community import community_louvain
import matplotlib.pyplot as plt
from networkx.algorithms import community
import pyvis
from pyvis import network as net
from graph_tool.all import *
import polars as pl
import json

## Class

In [4]:
# Classes and Functions Definition
class ComponentStructure:
    
    def __init__(self, graph, partition: dict):
        self.graph = nx.Graph.copy(graph)
        self.partition = partition
        self.communities_id = self.__get_communities()
        self.local_components = []
        self.global_component = {}
        self.lc_density = {}
        
    def set_local_components(self):
        for community_id in self.communities_id:
            nodes = set([k for k, v in self.partition.items() if v == community_id])
            part = {k: v for k, v in self.partition.items() if v == community_id}
            local_component = {"lc": self.graph.subgraph(nodes), "partition": part}
            self.local_components.append(local_component) 
        return
    
    def remove_intra_community_edges(self):
        temp_graph = nx.Graph.copy(self.graph)
        for community_id in self.communities_id:
            nodes = set([k for k, v in self.partition.items() if v == community_id])
            for node in nodes:
                neighbors = set(nx.neighbors(temp_graph, node))
                inner_nodes = neighbors.intersection(nodes)
                for inner_node in inner_nodes:
                    temp_graph.remove_edge(node, inner_node)
        temp_graph.remove_nodes_from(list(nx.isolates(temp_graph)))
        part = {k: v for k, v in self.partition.items() if k in temp_graph.nodes()}
        self.global_component = {'gp':temp_graph, 'part':part}
        return 
    
    def local_components_density(self):
        i = 0;
        for local_component in self.local_components:
            self.lc_density["lc"+str(i)] = nx.density(local_component["lc"])
            i = i+1
        return self.local_components_density
    
    def __get_communities(self) -> set:
        return set(self.partition.values())

## Functions

In [5]:
def drawer(net: pyvis.network, graph: nx.Graph(),  fixed: bool, partition=None):
    if( partition != None):
        for node in graph.nodes:
            graph.nodes[node]['group'] = partition[node]
            graph.nodes[node]['label'] = str(partition[node])
    nt = net.Network('500px', '500px')
    grid = net.Network(notebook=True)
    grid.from_nx(graph)
    grid.toggle_physics(fixed)
    return grid.show("test.html")

def drawer_lc(net: pyvis.network, graph: nx.Graph(),  fixed: bool, community_id=None, color=None):
    if( partition != None):
        for node in graph.nodes:
            graph.nodes[node]['color'] = color
            graph.nodes[node]['label'] = str(community_id)
    nt = net.Network('500px', '500px')
    grid = net.Network(notebook=True)
    grid.from_nx(graph)
    grid.toggle_physics(fixed)
    return grid.show("test.html")

def important_of_node(graph, node):
    groupes = []
    list_of_neighbors = list(nx.neighbors(graph, node))
    graph_data = dict(graph.nodes.data())
    neighbors_data = {k:v for k, v in graph_data.items() if k in list_of_neighbors}
    
    for k in neighbors_data.keys():
        groupes.append(neighbors_data[k]['group'])
    communities_groupes = list(set(groupes))
    #same_communities_neighbors = {k:v for k, v in neighbors_data.items() if v['group'] == communities_groupes[0]}
    number_of_nodes = len(communities_groupes)
    graph.nodes[node]['size'] = 10*number_of_nodes
    return 

def core_density(graph, partition):
    periphery_nodes = list({k:v for k, v in partition.items() if v == 0})
    temp_graph = nx.Graph.copy(graph)
    temp_graph.remove_nodes_from(periphery_nodes)
    return nx.density(temp_graph)

def periphery_density(graph, partition):
    periphery_nodes = list({k:v for k, v in partition.items() if v == 1})
    temp_graph = nx.Graph.copy(graph)
    temp_graph.remove_nodes_from(periphery_nodes)
    return nx.density(temp_graph)

def core_periphery_density(graph, partition):
    core_nodes = list({k:v for k, v in partition.items() if v == 1})
    periphery_nodes = list({k:v for k, v in partition.items() if v == 0})
    number_of_nc = len(core_nodes)
    number_of_np = len(periphery_nodes)
    temp_graph = nx.Graph.copy(graph)
    for node in core_nodes:
        neighbors = set(nx.neighbors(graph, node))
        inner_nodes = neighbors.intersection(core_nodes)
        for inner_node in inner_nodes:
            try:
                temp_graph.remove_edge(node, inner_node)
            except:
                pass
    density = len(temp_graph)/(number_of_nc*number_of_np)
    return density

def densities(graph, partition):
    return {
    "core-core": core_density(graph, partition),
    "core-periphery": core_periphery_density(graph, partition),
    "periphery-periphery": periphery_density(graph, partition) }

def json_writer(data, path, file_name):
    file = open(path+"/"+file_name+".json", "w")
    json.dump(data, file, indent=6)
    file.close()

### Load Graph

In [6]:
# Load data
fil_name = 'data'
data = pd.read_table(fil_name, sep='\t+', header=0, names=["source", "target"])
data

/tmp/ipykernel_172706/539008204.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_table(fil_name, sep='\t+', header=0, names=["source", "target"])


source  target
0           1       2
1           1       3
2           1       4
3           1       5
4           1       6
...       ...     ...
33425     975     664
33426     975      67
33427     975    1004
33428     975    1224
33429    1028     791

[33430 rows x 2 columns]

In [8]:
Root_graph = nx.from_pandas_edgelist(data, source='source', target='target', create_using=nx.Graph())
print("Number of edges before sanitize "+str(len(Root_graph.edges())))
print("Number of nodes before sanitize "+str(len(Root_graph.nodes())))

Number of edges before sanitize 16715
Number of nodes before sanitize 1224


In [9]:
selfloop_edges = list(nx.selfloop_edges(Root_graph))
print("We have  "+str(len(selfloop_edges))+ " selfloops in the graph")
Root_graph.remove_edges_from(selfloop_edges)
selfloop_edges = list(nx.selfloop_edges(Root_graph))
print("We have  "+str(len(selfloop_edges))+ " selfloops in the graph")

isolates_nodes = list(nx.isolates(Root_graph))
print("We have  "+str(len(isolates_nodes))+ " isolates nodes in the graph")
Root_graph.remove_nodes_from(isolates_nodes)
isolates_nodes = list(nx.isolates(Root_graph))
print("We have  "+str(len(isolates_nodes))+ " isolates nodes in the graph")
print("Number of edges after sanitize "+str(len(Root_graph.edges())))
print("Number of nodes after sanitize "+str(len(Root_graph.nodes())))

We have  0 selfloops in the graph
We have  0 selfloops in the graph
We have  0 isolates nodes in the graph
We have  0 isolates nodes in the graph
Number of edges after sanitize 16715
Number of nodes after sanitize 1224


In [ ]:
#Store graph as file
graph_json = nx.node_link_data(Root_graph)
json_writer(graph_json, "../data", "co_authorship_graph")